In [ ]:
import os
import time
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# Функция для проверки наличия новых файлов в папке
def check_for_new_files(directory, processed_files_path):
    new_files = []
    if not os.path.exists(processed_files_path):
        with open(processed_files_path, 'w') as f:
            pass
    with open(processed_files_path, 'r') as f:
        processed_files = set(f.read().splitlines())
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        if os.path.isfile(file_path) and file_path not in processed_files:
            new_files.append(file_path)
    with open(processed_files_path, 'a') as f:
        for file_path in new_files:
            f.write(file_path + '\n')
    return new_files

#------------------------------------------------------------------------------------------------------
# Тут нужно реализовать предобработку данных + добавить pronation_0, pronation_1, pronation_2 с помощью той лог.модели Олега
# Ниже чисто примерчик
def preprocess_data(file_path, features):
    
    data = pd.read_csv(file_path, index_col=0)
    X = data[features]
    scaler = MinMaxScaler()
    X_scaled = pd.DataFrame(
        scaler.fit_transform(X),
        columns=X.columns
    )
    return X_scaled
#------------------------------------------------------------------------------------------------------

# Функция для создания последовательностей
def create_sequences(data, timesteps=2):
    X = []
    for i in range(len(data) - timesteps + 1):
        X.append(data[i:i + timesteps])
    return np.array(X)

# Функция для предсказания жеста руки
def predict(model, X_test_seq):
    y_pred_encoded = model.predict(X_test_seq)
    y_pred = np.argmax(y_pred_encoded, axis=1)
    return y_pred


# Функция для постобработки и записи результатов в файл
def postprocess_and_write_to_file(predicted_gestures, file_path='logs/results.txt'):
    with open(file_path, 'a') as f:
        for gesture in predicted_gestures:
            # Возвращаем остаток от деления на 10
            processed_gesture = gesture % 10
            f.write(f"Predicted gesture: {processed_gesture}\n")



# Путь к папке с новыми данными и файлу с обработанными файлами
working_directory = 'working/files/'
processed_files_path = 'working/processed_files.txt'

# Путь к предобученной модели
model_path = 'motorica/model/lstm_model.h5'

# Загрузка предобученной модели
model = load_model(model_path)


# Основной цикл для мониторинга папки и обработки новых данных
while True:
    start_time = time.time()

    new_files = check_for_new_files(working_directory, processed_files_path)
    for file_path in new_files:
        print(f"Processing new file: {file_path}")
        # Предобработка новых данных
        features = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
                    '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', 
                    '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', 'ACC0', 'ACC1', 'ACC2']
        X_scaled = preprocess_data(file_path, features)
        X_seq = create_sequences(X_scaled.values)

        # Предсказание жеста руки 
        # Тут мы предсказываем 0..5, 10..15, 20..25
        predicted_gestures = predict(model, X_seq)

        # Постобработка и запись результатов в файл
        postprocess_and_write_to_file(predicted_gestures)

    # Пауза перед следующей проверкой
    elapsed_time = time.time() - start_time
    sleep_time = max(0.033 - elapsed_time, 0)
    time.sleep(sleep_time)
